# Auswertung der Stickoxide

Die Stickoxide wurden von unserem Scraper erfasst. Der Endpunkt lautet `https://data.rbb-online.de/nox/data?from=2020-03-03T23%3A00%3A00Z&to=2020-03-19T22%3A59%3A59Z`.

Das Notebook lässt sich über folgenden Befehl ausführen:
```
docker run --rm -p 8889:8888 --ip=0.0.0.0 -v (pwd):/home/jovyan -e CHOWN_HOME=yes -e CHOWN_EXTRA_ARGS=-R --user=root jupyter/scipy-notebook
```

Alle Dependencies, die nicht im `jupyter/scipy-notebook`-Image definiert sind, werden in der nächsten Zeile installiert.

In [1]:
! conda install altair -y

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/linux-64::blas==2.9=openblas
  - conda-forge/linux-64::liblapacke==3.8.0=9_openblas
  - conda-forge/linux-64::matplotlib==3.0.3=py37_1
  - conda-forge/linux-64::numba==0.42.1=py37hf484d3e_0
  - conda-forge/linux-64::openblas==0.3.6=h6e990d7_2
- ^C
failed

CondaError: KeyboardInterrupt



In [2]:
import pandas as pd
import numpy as np
import altair as alt
import json
import pytz

from helpers import read_nox

## Einlesen als Dataframes

In [3]:
measurements = read_nox('./input/2020-02-01--2020-04-06.json')

# drop last day to make sure we only have complete days
measurements = measurements[measurements['datetime'].dt.dayofyear != measurements['datetime'].max().dayofyear]
measurements.head()

NameError: name 'json' is not defined

In [ ]:
# how much data do we have?
measurements.count()

In [ ]:
berlin = measurements[measurements['federal_state'] == 'Berlin']
brandenburg = measurements[measurements['federal_state'] == 'Brandenburg']

In [ ]:
berlin.groupby(['name', berlin['datetime'].dt.day]).mean()

## Plotten

Um einen Eindruck zu erhalten, wie unsere Daten für Berlin und Brandenburg aussehen, plotten wir das tägliche Mittel pro Station:

In [ ]:
berlin_daily_avg = berlin.set_index('datetime').groupby('name').resample('D').mean().reset_index()
berlin_daily_avg.head()

In [ ]:
alt.Chart(berlin_daily_avg).mark_line(interpolate='basis').encode(
    x='datetime:T',
    y='val:Q',
    color='name:N'
)

In [ ]:
brandenburg_daily_avg = brandenburg.set_index('datetime').groupby('name').resample('D').mean().reset_index()
brandenburg_daily_avg.head()

In [ ]:
alt.Chart(brandenburg_daily_avg).mark_line(interpolate='basis').encode(
    x='datetime:T',
    y='val:Q',
    color='name:N'
)

Es scheint nicht so, als ob sich für die letzten zwei Wochen ein klarer Trend abzeichnet. Was man jetzt noch prüfen kann:

- Wie war das Wetter in den letzten zwei Wochen?
- Wie sieht die Stickstoffbelastung während der Spitzenzeiten aus?

## Spitzenzeiten

Als Spitzenzeiten gelten Zeiten zwischen 6 und 9 Uhr sowie 16 und 19 Uhr ([siehe Wikipedia](https://de.wikipedia.org/wiki/Verkehrszeiten#Hauptverkehrszeit)).

In [ ]:
def in_rush_hour(df):
    # we have to reconvert it to get the correct mask across summer and winter time
    converted = df['datetime'].dt.tz_convert('Europe/Berlin')
    h = converted.dt.hour
    return df[((h >= 6) & (h < 9)) | ((h >= 16) & (h < 19))]

In [ ]:
berlin_rush_hour = in_rush_hour(berlin)
berlin_rush_hour = berlin_rush_hour.set_index('datetime').groupby('name').resample('D').mean().reset_index()

In [ ]:
alt.Chart(berlin_rush_hour).mark_line(interpolate='basis').encode(
    x='datetime:T',
    y='val:Q',
    color='name:N'
)

In [ ]:
brandenburg_rush_hour = in_rush_hour(brandenburg)
brandenburg_rush_hour = brandenburg_rush_hour.set_index('datetime').groupby('name').resample('D').mean().reset_index()

In [ ]:
alt.Chart(brandenburg_rush_hour).mark_line(interpolate='basis').encode(
    x='datetime:T',
    y='val:Q',
    color='name:N'
)

Die Werte sind zwar etwas höher (wie erwartet), aber auch hier ergibt sich keine deutliche Änderung in jüngerer Vergangenheit. Wir versuchen es noch mal mit dem drei- und siebentägigen Mittel:

In [ ]:
def n_day_avg(df, n_days):
    '''
    Calculates a rolling average over n days
    '''
    return df.set_index('datetime').resample('D').mean().rolling(n_days).mean().dropna().reset_index()

In [ ]:
three_day_avg = n_day_avg(berlin, 3)
seven_day_avg = n_day_avg(berlin, 7)

alt.hconcat(
    alt.Chart(three_day_avg).mark_line(interpolate='basis').encode(x='datetime', y='val'),
    alt.Chart(seven_day_avg).mark_line(interpolate='basis').encode(x='datetime', y='val')
)

Auch nicht.

Es gibt [Studien](https://www.sciencedirect.com/science/article/pii/S0048969717319988), die [versuchen](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3856232/) Verkehr mithilfe von Luftverschmutzung zu schetzen und auch wenn sie vielversprechend sind (also hohe Korellationen von Vorhersagen und tatsächlichen Werten zeigen), gibt es keine, die das zuverlässig auf einem engen Raum tun.

Die [Esa Animation](https://www.esa.int/ESA_Multimedia/Videos/2020/03/Coronavirus_nitrogen_dioxide_emissions_drop_over_Italy) hat einen räumlich sehr grobe Auflösung - ganz Europa - zur Grundlage genommen und darauf einen vierzehntägen Durchschnitt berechnet um den Rückgang von Stickoxid in Zusammenhang mit den Ausgangssperren zu zeigen.

## Außerhalb der Rush Hour

In [ ]:
be_not_in_rush_hour = berlin[~berlin.index.isin(berlin_rush_hour.index)]
be_not_in_rush_hour = be_not_in_rush_hour.set_index('datetime').groupby('name').resample('D').mean().reset_index()
be_not_in_rush_hour.head()

In [ ]:
alt.Chart(be_not_in_rush_hour).mark_line(interpolate='basis').encode(
    x='datetime:T',
    y='val:Q',
    color='name:N'
)

In [ ]:
bb_not_in_rush_hour = brandenburg[~brandenburg.index.isin(brandenburg_rush_hour.index)]
bb_not_in_rush_hour = bb_not_in_rush_hour.set_index('datetime').groupby('name').resample('D').mean().reset_index()
bb_not_in_rush_hour.head()

In [ ]:
alt.Chart(bb_not_in_rush_hour).mark_line(interpolate='basis').encode(
    x='datetime:T',
    y='val:Q',
    color='name:N'
)

## Lineare Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression

### Tägliches Mittel

In [ ]:
daily_avg = berlin.set_index('datetime').resample('D').mean().reset_index()
daily_avg.head()

In [ ]:
x = daily_avg['datetime'].values.astype('int64').reshape(-1, 1)
y = daily_avg['val'].values.reshape(-1, 1)
linear_regressor = LinearRegression()
linear_regressor.fit(x, y)
linear_regressor.score(x, y)

$R^2$ ist fast 0, also gibt es in unserem Datensatz kaum eine Korrelation zwischen Datum und Stickstoffausstoß,

In [ ]:
daily_avg.loc[:,'prediction'] = linear_regressor.predict(x)

In [ ]:
alt.Chart(daily_avg).mark_line().encode(x='datetime', y='prediction')

## Weitere Gedanken

- Es gibt eine Studie, die Feinstaub (PM2.5) mit einer bestimmten Menge an Zigaretten gleichsetzt. Faustregel: 22 µg PM2.5 $\approx$ 1 Zigarette [Link](http://berkeleyearth.org/air-pollution-and-cigarette-equivalence/)

- Rumgeschwanke von normalen Wochen / vergleichbar mit Geschwanke vor Homeoffice (stddev / Fluktuation)
- Wann haben die Leute auf Homeoffice gesetzt?

- Die drei niedrigsten Stickoxiddaten in Berlin waren letzte Woche (Christopher)

### TODO

- Man sollte die höchsten / niedrigsten Werte isolieren und die Effekte da messen
- Wir sollten schauen, ob wir für den gesamten Zeitraum Messwerte haben
- Die Zeit hat einen [Artikel über die langfristigen Klimaauswirkungen geschrieben](https://www.zeit.de/2020/14/emissionen-corona-krise-klimaschutz-treibhausgase-co2?utm_medium=sm&utm_campaign=ref&utm_content=zeitde_redpost+_link_sf&utm_source=twitter_zonaudev_int&wt_zmc=sm.int.zonaudev.twitter.ref.zeitde.redpost.link.sf), der möglicherweise interessant ist

## Export für Götz

In [ ]:
goetz = measurements.copy()
goetz['datetime'] = goetz['datetime'].dt.tz_convert('Europe/Berlin')

In [ ]:
def get_weekday(row):
    weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    return weekdays[row['datetime'].weekday()]

In [ ]:
daily_avg_since_feb = goetz.set_index('datetime').groupby(['id', 'name']).resample('24h').mean().reset_index()
daily_avg_since_feb['weekday'] = daily_avg_since_feb.apply(get_weekday, axis=1)
daily_avg_since_feb.to_csv('daily_avg_since_feb.tsv', sep='\t')

In [ ]:
daily_std_since_feb = goetz.set_index('datetime').groupby(['id', 'name']).resample('24h').std().reset_index()
daily_std_since_feb['weekday'] = daily_std_since_feb.apply(get_weekday, axis=1)
daily_std_since_feb.to_csv('daily_std_since_feb.tsv', sep='\t')

In [ ]:
weekly_avg = goetz.set_index('datetime').groupby(['id', 'name']).resample('1w').mean().reset_index()
weekly_avg.to_csv('weekly_avg_since_feb.tsv', sep='\t')

In [ ]:
weekly_stddev = goetz.set_index('datetime').groupby(['id', 'name']).resample('1w').std().reset_index()
weekly_stddev.to_csv('weekly_std_since_feb.tsv', sep='\t')

In [ ]:
goetz.to_csv('all_values_since_feb.tsv', sep='\t')